In [4]:
import numpy as np
from collections import defaultdict

In [65]:
ipa_seqs = [
    [1, 10, 5],
    [10, 25, 9],
]

whisper_seqs = [
    [20, 9, 2],
    [11, 13, 6, 10],
]

ipa_vocab_size = np.max(np.concatenate(ipa_seqs))+1
whisper_vocab_size = np.max(np.concatenate(whisper_seqs))+1

sum_sqd_dist = np.zeros([whisper_vocab_size, ipa_vocab_size])
ipa_token_counts = np.zeros(ipa_vocab_size)

In [66]:
sum_sqd_dist.shape

(21, 26)

In [67]:
np.expand_dims(ipa_token_counts, axis=0).shape

(1, 26)

In [72]:
def initialize_probs(ipa_seqs, whisper_seqs, sum_sqd_dist, ipa_token_counts):
    for ipa_seq, whisper_seq in zip(ipa_seqs, whisper_seqs):
        for i, ipa_token in enumerate(ipa_seq):
            ipa_token_counts[ipa_token]+=1
            for j, whisper_token in enumerate(whisper_seq):
                sum_sqd_dist[whisper_token,ipa_token]+=(i-j)**2
    prob_unnorm = np.matmul(sum_sqd_dist, ipa_token_counts)
    return prob_unnorm
    
initialize_probs(ipa_seqs, whisper_seqs, sum_sqd_dist, ipa_token_counts)

array([  0.,   0., 150.,   0.,   0.,   0., 225.,   0.,   0.,  50., 575.,
       125.,   0.,  75.,   0.,   0.,   0.,   0.,   0.,   0., 150.])